In [1]:
# !pip install --quiet pytorch_lightning
# !pip install --quiet transformers

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
# os.environ['TORCH_USE_CUDA_DSA'] = '0'

In [3]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from sklearn.model_selection import train_test_split
import textwrap
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

from transformers import(
    AdamW,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

In [4]:
from datasets import Dataset, load_metric

In [5]:
torch.cuda.current_device()

0

In [6]:
def get_device_and_set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    return device
    
SEED = 123
device = get_device_and_set_seed(SEED)

In [7]:
device

device(type='cuda')

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base", load_in_8bit=True, device_map="auto")
# model.to(device)
next(model.parameters()).is_cuda

In [7]:
model = AutoModelForCausalLM.from_pretrained("vilm/vinallama-2.7b", load_in_4bit=True, device_map="auto")
# model.to(device)
next(model.parameters()).is_cuda

True

In [8]:
tokenizer = AutoTokenizer.from_pretrained("vilm/vinallama-2.7b", legacy=False)  

In [9]:
labels = tokenizer(
        'tôi thích bạn', max_length=256, truncation=True, padding=True
    )

In [10]:
tokenizer.convert_ids_to_tokens(labels['input_ids'])

['<s>', '▁tôi', '▁thích', '▁bạn']

In [11]:
tokenizer

LlamaTokenizerFast(name_or_path='vilm/vinallama-2.7b', vocab_size=46303, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46303: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
labels

{'input_ids': [1, 32217, 32802, 32362], 'attention_mask': [1, 1, 1, 1]}

## Prepare Data

In [25]:
train_path = './data/address_data_1.csv'

In [26]:
data_df = pd.read_csv(train_path)
data_df.head()

,input_address,filter_address,mistake_address
0,"Thửa đất số 11, Tờ bản đồ số 39, Ấp Hoàng Việt...","Xã Tân Phước, Huyện Tân Hồng, Tỉnh Đồng Tháp",Thua dat so 11 To ban do so 39 Ap Hoang Viet X...
1,"Huyện Tân Hồng, Tỉnh Đồng Tháp","Huyện Tân Hồng, Tỉnh Đồng Tháp","uện n Hồng, ỉnh Đồng Tháp"
2,"Số 27, Đường Thiên Hộ Dương, Khóm 3, Phường An...","Đường Thiên Hộ Dương, Phường An Thạnh, Thành p...",So 27 Duong Thien Ho Duong Khom 3 Phuong An Th...
3,"Phường An Thạnh, Thành phố Hồng Ngự, Tỉnh Đồng...","Phường An Thạnh, Thành phố Hồng Ngự, Tỉnh Đồng...",hường An Thạn Tành phố Hồg Ngự Tỉh Đồng Tháp
4,"Tổ 20, Khóm An Lợi, Phường An Bình A, Thành ph...","Phường An Bình A, Thành phố Hồng Ngự, Tỉnh Đồn...","Tổ 20, Khóm An Loi, Phường An Bình A, Thành ph..."


In [10]:
data_df = data_df[:500000]
data_df

,input_address,filter_address,mistake_address
0,"Thửa đất số 11, Tờ bản đồ số 39, Ấp Hoàng Việt...","Xã Tân Phước, Huyện Tân Hồng, Tỉnh Đồng Tháp",Thua dat so 11 To ban do so 39 Ap Hoang Viet X...
1,"Huyện Tân Hồng, Tỉnh Đồng Tháp","Huyện Tân Hồng, Tỉnh Đồng Tháp","uện n Hồng, ỉnh Đồng Tháp"
2,"Số 27, Đường Thiên Hộ Dương, Khóm 3, Phường An...","Đường Thiên Hộ Dương, Phường An Thạnh, Thành p...",So 27 Duong Thien Ho Duong Khom 3 Phuong An Th...
3,"Phường An Thạnh, Thành phố Hồng Ngự, Tỉnh Đồng...","Phường An Thạnh, Thành phố Hồng Ngự, Tỉnh Đồng...",hường An Thạn Tành phố Hồg Ngự Tỉh Đồng Tháp
4,"Tổ 20, Khóm An Lợi, Phường An Bình A, Thành ph...","Phường An Bình A, Thành phố Hồng Ngự, Tỉnh Đồn...","Tổ 20, Khóm An Loi, Phường An Bình A, Thành ph..."
...,...,...,...
499995,"5D Tầng Trệt, Chung cư CT3, Khu đô thị VCN Phư...","Phường Phước Hải, Thành phố Nha Trang, Khánh Hòa",5D Tang Tret Chung cu CT3 Khu do thi VCN Phuoc...
499996,"50 Đại lộ Lê Lợi, Phường Tân Sơn, Thành phố Th...","Đại lộ Lê Lợi, Phường Tân Sơn, Thành phố Thanh...",50 Dai lo Le Loi Phuong Tan Son Thanh pho Than...
499997,"Phường Tân Sơn, Thành phố Thanh Hóa, Thanh Hóa","Phường Tân Sơn, Thành phố Thanh Hóa, Thanh Hóa",phường tân sơn thành phố thanh hóa thanh hóa
499998,"206 Huỳnh Tấn Phát, Phường Khuê Trung, Quận Cẩ...","Phường Khuê Trung, Quận Cẩm Lệ, Đà Nẵng","2 06 h uỳ nh tấn phát, phường khuê trung, quận..."


In [27]:
data_df = data_df.dropna()

In [28]:
train_df, test_df = train_test_split(data_df, test_size=0.1, random_state=SEED)

In [29]:
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=SEED)

In [30]:
meta_df = pd.read_csv('./data/address_data_meta_1.csv')
meta_df

,input_address,filter_address,mistake_address
0,Thành phố Hà Nội,Thành phố Hà Nội,thành phố hà nội
1,"Quận Ba Đình, Thành phố Hà Nội","Quận Ba Đình, Thành phố Hà Nội","Quận BaĐn, Tành phốHà Nội"
2,"Phường Phúc Xá, Quận Ba Đình, Thành phố Hà Nội","Phường Phúc Xá, Quận Ba Đình, Thành phố Hà Nội","Phường Phúc Xá, Quận Ba Dinh, Thành pho Ha Nội"
3,"Phường Trúc Bạch, Quận Ba Đình, Thành phố Hà Nội","Phường Trúc Bạch, Quận Ba Đình, Thành phố Hà Nội","hườg Trúc Bạch, Quận a Đình, Thàh phố Hà ội"
4,"Phường Vĩnh Phúc, Quận Ba Đình, Thành phố Hà Nội","Phường Vĩnh Phúc, Quận Ba Đình, Thành phố Hà Nội",hường Vĩnh Phúc Qun Ba Đìh Thành phố H ội
...,...,...,...
11379,"Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An Đông, Huyện Ngoc Hiển, Tinh Cà Mau"
11380,"Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau","X iên An, Huyện Ngọ iển,Tỉnh Cà Mau"
11381,"Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau","Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau",Thị trấn Rạch Gốc Huyện Ngoc Hien Tỉnh Ca Mau
11382,"Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau",Xã ân n HuyệnNgọ Hiển TỉnhCà Mau


In [31]:
train_df = pd.concat([train_df, meta_df], ignore_index=True)
train_df 

,input_address,filter_address,mistake_address
0,"NO01 - LK 41 khu dịch vụ HT5, KĐT Văn Khê, Phư...","Phường La Khê, Quận Hà Đông, Thành phố Hà Nội",NO01 LK 41 khu dich vu HT5 KDT Van Khe Phuong...
1,"102/7/8 Cống Quỳnh, Phường Phạm Ngũ Lão, Quận ...","Phường Phạm Ngũ Lão, Quận 1, Thành phố Hồ Chí ...","102/7/8 Cống Quỳnh, Phường Pham Ngu Lao, Quận ..."
2,"Số 28-30 đường Quang Trung, Khóm Châu Quới 3, ...","đường Quang Trung, Phường Châu Phú B, Thành ph...",Số 2830 duong Quang Trung Khóm Châu Quới 3 Phu...
3,"Thôn Thanh Đặng, Xã Minh Hải, Huyện Văn Lâm, H...","Xã Minh Hải, Huyện Văn Lâm, Hưng Yên","Thôn Than Đặng, Xã Minh Hi, Huyện Văn Lâm,Hư Yên"
4,"Số 198 Bùi Thị Xuân, Phường 1, Thành phố Bảo L...","Phường 1, Thành phố Bảo Lộc, Tỉnh Lâm Đồng",So 198 Bùi Thị Xuan Phường 1 Thành phố Bao Lộc...
...,...,...,...
889451,"Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An Đông, Huyện Ngoc Hiển, Tinh Cà Mau"
889452,"Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau","X iên An, Huyện Ngọ iển,Tỉnh Cà Mau"
889453,"Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau","Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau",Thị trấn Rạch Gốc Huyện Ngoc Hien Tỉnh Ca Mau
889454,"Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau",Xã ân n HuyệnNgọ Hiển TỉnhCà Mau


In [32]:
train_df

,input_address,filter_address,mistake_address
0,"NO01 - LK 41 khu dịch vụ HT5, KĐT Văn Khê, Phư...","Phường La Khê, Quận Hà Đông, Thành phố Hà Nội",NO01 LK 41 khu dich vu HT5 KDT Van Khe Phuong...
1,"102/7/8 Cống Quỳnh, Phường Phạm Ngũ Lão, Quận ...","Phường Phạm Ngũ Lão, Quận 1, Thành phố Hồ Chí ...","102/7/8 Cống Quỳnh, Phường Pham Ngu Lao, Quận ..."
2,"Số 28-30 đường Quang Trung, Khóm Châu Quới 3, ...","đường Quang Trung, Phường Châu Phú B, Thành ph...",Số 2830 duong Quang Trung Khóm Châu Quới 3 Phu...
3,"Thôn Thanh Đặng, Xã Minh Hải, Huyện Văn Lâm, H...","Xã Minh Hải, Huyện Văn Lâm, Hưng Yên","Thôn Than Đặng, Xã Minh Hi, Huyện Văn Lâm,Hư Yên"
4,"Số 198 Bùi Thị Xuân, Phường 1, Thành phố Bảo L...","Phường 1, Thành phố Bảo Lộc, Tỉnh Lâm Đồng",So 198 Bùi Thị Xuan Phường 1 Thành phố Bao Lộc...
...,...,...,...
889451,"Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An Đông, Huyện Ngoc Hiển, Tinh Cà Mau"
889452,"Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau","X iên An, Huyện Ngọ iển,Tỉnh Cà Mau"
889453,"Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau","Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau",Thị trấn Rạch Gốc Huyện Ngoc Hien Tỉnh Ca Mau
889454,"Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau","Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau",Xã ân n HuyệnNgọ Hiển TỉnhCà Mau


In [34]:
val_df

,input_address,filter_address,mistake_address
686189,"49/46/146 Đường Số 51, Phường 14, Quận Gò Vấp,...","Đường Số 51, Phường 14, Quận Gò Vấp, TP Hồ Chí...","49/46 /146 đường số 51, phường 14, q uận gò ..."
1046878,"203 Cư xá Tân Sơn Nhì 1, Ba Vân, Phường 14, Qu...","Phường 14, Quận Tân Bình, TP Hồ Chí Minh","203 cư xá tân sơn nhì 1, ba vân, phường 14, ..."
329395,"Khối Trung Thành, Phường Nghi Hải, Thị xã Cửa ...","Phường Nghi Hải, Thị xã Cửa Lò, Tỉnh Nghệ An","Khối Tung Thàh, Phường Nghi ải, Thị xã Cửa Lò,..."
167820,"Số 22 phố Trần Khát Chân, Phường Bạch Đằng, Qu...","Phường Bạch Đằng, Quận Hai Bà Trưng, Thành phố...",số 22 phố trần khát chân phường bạch đằng quận...
341349,"Số nhà 10, Ngách 102/27 Khuất Duy Tiến, Phường...","Phường Nhân Chính, Quận Thanh Xuân, Thành phố ...",số nhà 10 ngách 102/27 khuất duy tiến phường n...
...,...,...,...
417869,"Số 399, đường Nguyễn Xiển, Phường Đại Kim, Quậ...","đường Nguyễn Xiển, Phường Đại Kim, Quận Hoàng ...",Số 399 đường Nguyễn Xiển Phuong Đại Kim Quan H...
493172,"Tổ 1, ấp 2, Xã Suối Ngô, Huyện Tân Châu, Tây Ninh","Xã Suối Ngô, Huyện Tân Châu, Tây Ninh","tổ 1, ấp 2, xã suối ngô, huyện tân ch âu, t..."
173900,"240 Nguyễn Huệ, Thị Trấn La Hà, Huyện Tư Nghĩa...","Thị Trấn La Hà, Huyện Tư Nghĩa, Tỉnh Quảng Ngãi",240 nguyễn huệ thị trấn la hà huyện tư nghĩa t...
680769,"Lô E5, KCN Cầu Tràm, Xã Long Trạch, Huyện Cần ...","Xã Long Trạch, Huyện Cần Đước, Long An",lô e5 kcn cầu tràm xã long trạch huyện cần đướ...


In [22]:
def preprocess_function(examples, padding="max_length"):
    # tokenize inputs
    model_inputs = tokenizer(
        examples["inputs"], max_length=256, truncation=True, padding=True
    )
    
    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(
        examples["labels"], max_length=256, truncation=True, padding=True
    )
    
    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
        
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [21]:
dict_obj = {'inputs': train_df['mistake_address'], 'labels': train_df['filter_address']}
dataset = Dataset.from_dict(dict_obj)
# dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

Map (num_proc=8):   0%|          | 0/415245 [00:00<?, ? examples/s]

/home/nghiatl/anaconda3/envs/chatgpt/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/nghiatl/anaconda3/envs/chatgpt/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=8):   0%|          | 0/46139 [00:00<?, ? examples/s]

In [ ]:
dict_obj = {'inputs': train_df['mistake_address'], 'labels': train_df['filter_address']}
dataset = Dataset.from_dict(dict_obj)
# dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [22]:
dict_obj = {'inputs': test_df['mistake_address'], 'labels': test_df['mistake_address']}
dataset = Dataset.from_dict(dict_obj)
test_data = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

Map (num_proc=8):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [24]:
# train_data['train'].__getitem__(0)['input_ids']

In [25]:
train_data['train'].column_names

['labels', 'input_ids', 'attention_mask']

In [24]:
len(test_data)

50000

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, pad_to_multiple_of=8, return_tensors="pt")
# data_collator

In [32]:
data_collator([train_data['train'].__getitem__(2)])['labels'].shape

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([1, 32])

## PEFT

In [33]:
from peft import PeftModel, LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

In [51]:
from trl import SFTTrainer

In [38]:
# Define LoRA Config
lora_config = LoraConfig(
 r=8,
 lora_alpha=16,
 target_modules=["q_proj", "v_proj"],
 lora_dropout=0.05,
 bias="none",
 task_type="CAUSAL_LM"
)
# prepare int-8 model for training
# model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,621,440 || all params: 2,777,418,240 || trainable%: 0.09438405646821128


In [26]:
training_args = Seq2SeqTrainingArguments(
    "T5_address_model/",
    do_train=True,
    do_eval=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=15,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    logging_dir='./log',
    group_by_length=True,
    load_best_model_at_end=True,
    save_total_limit=1,
    fp16=True,
)

In [40]:
output_dir = "lora_address_model"

In [46]:
training_args = Seq2SeqTrainingArguments(
    output_dir="lora_llama_address_model/",
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1, 
    learning_rate=1e-4, # higher learning rate
    num_train_epochs=3,
    logging_dir=f"{output_dir}/logs",
    logging_steps = 500,
    group_by_length=True,
    save_strategy='epoch',
    # load_best_model_at_end=True,
    save_total_limit=2,
)

In [47]:
len(train_data["train"]) / 4

1125.0

In [48]:
300 * 5

1500

## Training

In [44]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [57]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data["train"],
    eval_dataset=train_data["test"],
    peft_config=lora_config,
    packing=True,
    tokenizer=tokenizer,
    args=training_args
)

ValueError: You need to pass a `dataset_text_field` or `formatting_func` argument to the SFTTrainer if you want to use the `ConstantLengthDataset`.

In [45]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data["train"],
    eval_dataset=train_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Currently logged in as: lenghia11a4. Use `wandb login --relogin` to force relogin


/home/nghiatl/anaconda3/envs/chatgpt/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


ValueError: Expected input batch_size (376) to match target batch_size (248).

In [ ]:
model

## Inference

In [40]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base", load_in_8bit=True, device_map="auto")

In [41]:
peft_model_id = './lora_T5_address_model/checkpoint-1510'

In [42]:
peft_model = PeftModel.from_pretrained(model, peft_model_id)

### Test 

In [60]:
idx = 4
test_df.iloc[idx]['mistake_address']

'1/4L đường 74 Phường Phước Long A Thành hố Thủ Đchàhphố Hồ Chí Minh'

In [61]:
t = test_df.iloc[idx]['mistake_address']
b = tokenizer(t, return_tensors='pt')
b

{'input_ids': tensor([[30981,   547,   355,  2241, 11158,  2879,  2241,  2879,  8195,  5950,
           298,   259, 26902,   382,  1263,  1516,   708,   977,   697,   369,
           334,  6734,  1263,   447,  2434,  1536,   420, 14098,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}

In [62]:
outputs = model.generate(
      input_ids=b['input_ids'].to('cuda'),
      max_length=256,
      attention_mask=b['attention_mask'].to('cuda'),
  )
outputs

tensor([[     0, 250099,   2879,   8195,   5950,    261,    447,   2434,   1536,
            420,  14098,    261,      1]], device='cuda:0')

In [63]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'<extra_id_0> Phước Long, Hồ Chí Minh,'

### Eval metrics

In [52]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [57]:
import tqdm
import torch 
import numpy as np
metrics = rouge

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_data, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(dataloader):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

  labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
  actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

{'rouge1': 0.23835126750408075,
 'rouge2': 0.14379510590522415,
 'rougeL': 0.2161130252997705,
 'rougeLsum': 0.21546445876754589}

In [26]:
correct = 0
correct += sum(o==a for o, a in zip(predictions, references))
correct

5341

In [27]:
correct/ len(predictions)

0.9945996275605214

In [28]:
predictions[0]

'nguyễn văn tiến'

In [29]:
references[0]

'nguyễn văn tiến'

In [30]:
a= next(iter(dataloader))

In [31]:
tokenizer.decode(a['input_ids'][0], skip_special_tokens=True)

'nguyễn văn tiến thì dạ bên không cho'